In [1]:
import numpy as np
import warnings 
import sys 
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/andro.arff')
data = pd.DataFrame(data)
data.head()

,Window0-Att0,Window0-Att1,Window0-Att2,Window0-Att3,Window0-Att4,Window0-Att5,Window1-Att0,Window1-Att1,Window1-Att2,Window1-Att3,...,Window4-Att2,Window4-Att3,Window4-Att4,Window4-Att5,Target,Target_2,Target_3,Target_4,Target_5,Target_6
0,11.84,5.08,49.0,32.2,16.5,1.40,12.01,5.09,48.0,32.0,...,45.0,29.7,38.5,3.27,15.17,5.14,46.0,30.4,46.8,3.74
1,12.01,5.09,48.0,32.0,14.8,1.25,12.89,5.10,48.0,31.9,...,46.0,30.1,42.6,3.55,14.85,5.11,47.0,31.2,46.8,3.78
2,12.89,5.10,48.0,31.9,8.6,0.71,13.95,5.11,47.0,30.8,...,48.0,31.3,46.0,3.79,14.72,5.10,48.0,31.7,42.7,3.40
3,13.95,5.11,47.0,30.8,20.2,1.64,13.75,5.13,45.0,29.7,...,47.0,31.1,46.7,3.79,15.28,5.11,48.0,32.0,42.5,3.38
4,13.75,5.13,45.0,29.7,38.5,3.27,13.34,5.13,46.0,30.1,...,45.0,29.7,45.1,3.75,16.15,5.13,47.0,30.8,42.1,3.27


In [3]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=100)

(49, 30) (49, 6)


In [4]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(34, 30) (34, 6) (15, 30) (15, 6)


In [5]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 49
    p = 30

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

### Bidirectional Model

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [7]:
ENS_COUNT = 8

In [8]:
def create_c1node(X_feed,y_now):
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='he_normal'))
    model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='he_normal'))
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
    
    return model

### Forward Ensemble

In [ ]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_forward = []
    
    X_feed = X_train.copy()
    y_t = y_train.copy()
    
    for i in range(y_t.shape[1]):
        print('Training chain node ',i)
        y_now = y_t[:,[i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_forward.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_fwd.append(chain_forward)

Training Ensemble 0 - 

Training chain node  0
Shapes - 
 X = (34, 30) 
 Y = (34, 1)





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
34/34 [==============================] - 7s - loss: 1.8118 - mean_squared_error: 1.8118
Epoch 2/500
34/34 [==============================] - 0s - loss: 1.1674 - mean_squared_error: 1.1674
Epoch 3/500
34/34 [==============================] - 0s - loss: 0.7657 - mean_squared_error: 0.7657
Epoch 4/500
34/34 [==============================] - 0s - loss: 0.5759 - mean_squared_error: 0.5759
Epoch 5/500
34/34 [==============================] - 0s - loss: 0.5198 - mean_squared_error: 0.5198
Epoch 6/500
34/34 [==============================] - 0s - loss: 0.5182 - mean_squared_error: 0.5182
Epoch 7/500
34/34 [==============================] - 0s - loss: 0.5112 - mean_squared_error: 0.5112
Epoch 8/500
34/34 [==============================] - 0s - loss: 0.4798 - mean_squared_error

34/34 [==============================] - 0s - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 67/500
34/34 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027
Epoch 68/500
34/34 [==============================] - 0s - loss: 0.0024 - mean_squared_error: 0.0024
Epoch 69/500
34/34 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023
Epoch 70/500
34/34 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 71/500
34/34 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 72/500
34/34 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 73/500
34/34 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020
Epoch 74/500
34/34 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019
Epoch 75/500
34/34 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018
Epo

34/34 [==============================] - 0s - loss: 1.7471e-04 - mean_squared_error: 1.7471e-04
Epoch 143/500
34/34 [==============================] - 0s - loss: 1.6956e-04 - mean_squared_error: 1.6956e-04
Epoch 144/500
34/34 [==============================] - 0s - loss: 1.6458e-04 - mean_squared_error: 1.6458e-04
Epoch 145/500
34/34 [==============================] - 0s - loss: 1.5978e-04 - mean_squared_error: 1.5978e-04
Epoch 146/500
34/34 [==============================] - 0s - loss: 1.5543e-04 - mean_squared_error: 1.5543e-04
Epoch 147/500
34/34 [==============================] - 0s - loss: 1.5101e-04 - mean_squared_error: 1.5101e-04
Epoch 148/500
34/34 [==============================] - 0s - loss: 1.4660e-04 - mean_squared_error: 1.4660e-04
Epoch 149/500
34/34 [==============================] - 0s - loss: 1.4221e-04 - mean_squared_error: 1.4221e-04
Epoch 150/500
34/34 [==============================] - 0s - loss: 1.3785e-04 - mean_squared_error: 1.3785e-04
Epoch 151/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.6297e-05 - mean_squared_error: 1.6297e-05
Epoch 217/500
34/34 [==============================] - 0s - loss: 1.5721e-05 - mean_squared_error: 1.5721e-05
Epoch 218/500
34/34 [==============================] - 0s - loss: 1.5165e-05 - mean_squared_error: 1.5165e-05
Epoch 219/500
34/34 [==============================] - 0s - loss: 1.4631e-05 - mean_squared_error: 1.4631e-05
Epoch 220/500
34/34 [==============================] - 0s - loss: 1.4118e-05 - mean_squared_error: 1.4118e-05
Epoch 221/500
34/34 [==============================] - 0s - loss: 1.3614e-05 - mean_squared_error: 1.3614e-05
Epoch 222/500
34/34 [==============================] - 0s - loss: 1.3120e-05 - mean_squared_error: 1.3120e-05
Epoch 223/500
34/34 [==============================] - 0s - loss: 1.2654e-05 - mean_squared_error: 1.2654e-05
Epoch 224/500
34/34 [==============================] - 0s - loss: 1.2197e-05 - mean_squared_error: 1.2197e-05
Epoch 225/500
34/34 [===

34/34 [==============================] - 0s - loss: 9.9605e-07 - mean_squared_error: 9.9605e-07
Epoch 291/500
34/34 [==============================] - 0s - loss: 9.5515e-07 - mean_squared_error: 9.5515e-07
Epoch 292/500
34/34 [==============================] - 0s - loss: 9.1597e-07 - mean_squared_error: 9.1597e-07
Epoch 293/500
34/34 [==============================] - 0s - loss: 8.7751e-07 - mean_squared_error: 8.7751e-07
Epoch 294/500
34/34 [==============================] - 0s - loss: 8.4117e-07 - mean_squared_error: 8.4117e-07
Epoch 295/500
34/34 [==============================] - 0s - loss: 8.0720e-07 - mean_squared_error: 8.0720e-07
Epoch 296/500
34/34 [==============================] - 0s - loss: 7.7375e-07 - mean_squared_error: 7.7375e-07
Epoch 297/500
34/34 [==============================] - 0s - loss: 7.4165e-07 - mean_squared_error: 7.4165e-07
Epoch 298/500
34/34 [==============================] - 0s - loss: 7.1121e-07 - mean_squared_error: 7.1121e-07
Epoch 299/500
34/34 [===

...

34/34 [==============================] - 0s - loss: 2.2835e-14 - mean_squared_error: 2.2835e-14
Epoch 335/500
34/34 [==============================] - 0s - loss: 3.0816e-14 - mean_squared_error: 3.0816e-14
Epoch 336/500
34/34 [==============================] - 0s - loss: 2.8684e-14 - mean_squared_error: 2.8684e-14
Epoch 337/500
34/34 [==============================] - 0s - loss: 2.8089e-14 - mean_squared_error: 2.8089e-14
Epoch 338/500
34/34 [==============================] - 0s - loss: 2.2548e-14 - mean_squared_error: 2.2548e-14
Epoch 339/500
34/34 [==============================] - 0s - loss: 2.0415e-14 - mean_squared_error: 2.0415e-14
Epoch 340/500
34/34 [==============================] - 0s - loss: 2.0789e-14 - mean_squared_error: 2.0789e-14
Epoch 341/500
34/34 [==============================] - 0s - loss: 1.7158e-14 - mean_squared_error: 1.7158e-14
Epoch 342/500
34/34 [==============================] - 0s - loss: 1.7295e-14 - mean_squared_error: 1.7295e-14
Epoch 343/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.2819e-14 - mean_squared_error: 1.2819e-14
Epoch 409/500
34/34 [==============================] - 0s - loss: 1.0050e-14 - mean_squared_error: 1.0050e-14
Epoch 410/500
34/34 [==============================] - 0s - loss: 1.0650e-14 - mean_squared_error: 1.0650e-14
Epoch 411/500
34/34 [==============================] - 0s - loss: 9.2071e-15 - mean_squared_error: 9.2071e-15
Epoch 412/500
34/34 [==============================] - 0s - loss: 9.1353e-15 - mean_squared_error: 9.1353e-15
Epoch 413/500
34/34 [==============================] - 0s - loss: 8.5895e-15 - mean_squared_error: 8.5895e-15
Epoch 414/500
34/34 [==============================] - 0s - loss: 8.6940e-15 - mean_squared_error: 8.6940e-15
Epoch 415/500
34/34 [==============================] - 0s - loss: 9.2949e-15 - mean_squared_error: 9.2949e-15
Epoch 416/500
34/34 [==============================] - 0s - loss: 1.0510e-14 - mean_squared_error: 1.0510e-14
Epoch 417/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.3855e-14 - mean_squared_error: 1.3855e-14
Epoch 483/500
34/34 [==============================] - 0s - loss: 1.3588e-14 - mean_squared_error: 1.3588e-14
Epoch 484/500
34/34 [==============================] - 0s - loss: 1.2549e-14 - mean_squared_error: 1.2549e-14
Epoch 485/500
34/34 [==============================] - 0s - loss: 1.2699e-14 - mean_squared_error: 1.2699e-14
Epoch 486/500
34/34 [==============================] - 0s - loss: 1.0497e-14 - mean_squared_error: 1.0497e-14
Epoch 487/500
34/34 [==============================] - 0s - loss: 8.1197e-15 - mean_squared_error: 8.1197e-15
Epoch 488/500
34/34 [==============================] - 0s - loss: 1.1357e-14 - mean_squared_error: 1.1357e-14
Epoch 489/500
34/34 [==============================] - 0s - loss: 1.0752e-14 - mean_squared_error: 1.0752e-14
Epoch 490/500
34/34 [==============================] - 0s - loss: 1.2304e-14 - mean_squared_error: 1.2304e-14
Epoch 491/500
34/34 [===

34/34 [==============================] - 0s - loss: 0.0187 - mean_squared_error: 0.0187
Epoch 61/500
34/34 [==============================] - 0s - loss: 0.0178 - mean_squared_error: 0.0178
Epoch 62/500
34/34 [==============================] - 0s - loss: 0.0170 - mean_squared_error: 0.0170
Epoch 63/500
34/34 [==============================] - 0s - loss: 0.0162 - mean_squared_error: 0.0162
Epoch 64/500
34/34 [==============================] - 0s - loss: 0.0156 - mean_squared_error: 0.0156
Epoch 65/500
34/34 [==============================] - 0s - loss: 0.0152 - mean_squared_error: 0.0152
Epoch 66/500
34/34 [==============================] - 0s - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 67/500
34/34 [==============================] - 0s - loss: 0.0145 - mean_squared_error: 0.0145
Epoch 68/500
34/34 [==============================] - 0s - loss: 0.0140 - mean_squared_error: 0.0140
Epoch 69/500
34/34 [==============================] - 0s - loss: 0.0135 - mean_squared_error: 0.0135
Epo

34/34 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 141/500
34/34 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 142/500
34/34 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 143/500
34/34 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 144/500
34/34 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 145/500
34/34 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012
Epoch 146/500
34/34 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012
Epoch 147/500
34/34 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012
Epoch 148/500
34/34 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011
Epoch 149/500
34/34 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0

### Backward Ensemble

In [10]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_reverse = []
    
    X_feed = X_train
    y_t = y_train
    for i in range(1,y_t.shape[1]+1):
        print('Training data - ',i)
        y_now = y_t[:,[-i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_reverse.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_rev.append(chain_reverse)

34/34 [==============================] - 0s - loss: 3.5160e-14 - mean_squared_error: 3.5160e-14
Epoch 435/500
34/34 [==============================] - 0s - loss: 3.2532e-14 - mean_squared_error: 3.2532e-14
Epoch 436/500
34/34 [==============================] - 0s - loss: 2.9890e-14 - mean_squared_error: 2.9890e-14
Epoch 437/500
34/34 [==============================] - 0s - loss: 2.3542e-14 - mean_squared_error: 2.3542e-14
Epoch 438/500
34/34 [==============================] - 0s - loss: 2.7026e-14 - mean_squared_error: 2.7026e-14
Epoch 439/500
34/34 [==============================] - 0s - loss: 2.3268e-14 - mean_squared_error: 2.3268e-14
Epoch 440/500
34/34 [==============================] - 0s - loss: 2.3379e-14 - mean_squared_error: 2.3379e-14
Epoch 441/500
34/34 [==============================] - 0s - loss: 2.2882e-14 - mean_squared_error: 2.2882e-14
Epoch 442/500
34/34 [==============================] - 0s - loss: 2.3927e-14 - mean_squared_error: 2.3927e-14
Epoch 443/500
34/34 [===

34/34 [==============================] - 0s - loss: 0.2697 - mean_squared_error: 0.2697
Epoch 9/500
34/34 [==============================] - 0s - loss: 0.2040 - mean_squared_error: 0.2040
Epoch 10/500
34/34 [==============================] - 0s - loss: 0.1568 - mean_squared_error: 0.1568
Epoch 11/500
34/34 [==============================] - 0s - loss: 0.1313 - mean_squared_error: 0.1313
Epoch 12/500
34/34 [==============================] - 0s - loss: 0.1229 - mean_squared_error: 0.1229
Epoch 13/500
34/34 [==============================] - 0s - loss: 0.1229 - mean_squared_error: 0.1229
Epoch 14/500
34/34 [==============================] - 0s - loss: 0.1232 - mean_squared_error: 0.1232
Epoch 15/500
34/34 [==============================] - 0s - loss: 0.1194 - mean_squared_error: 0.1194
Epoch 16/500
34/34 [==============================] - 0s - loss: 0.1108 - mean_squared_error: 0.1108
Epoch 17/500
34/34 [==============================] - 0s - loss: 0.0988 - mean_squared_error: 0.0988
Epoc

34/34 [==============================] - 0s - loss: 4.1330e-04 - mean_squared_error: 4.1330e-04
Epoch 88/500
34/34 [==============================] - 0s - loss: 3.9508e-04 - mean_squared_error: 3.9508e-04
Epoch 89/500
34/34 [==============================] - 0s - loss: 3.7670e-04 - mean_squared_error: 3.7670e-04
Epoch 90/500
34/34 [==============================] - 0s - loss: 3.5892e-04 - mean_squared_error: 3.5892e-04
Epoch 91/500
34/34 [==============================] - 0s - loss: 3.4238e-04 - mean_squared_error: 3.4238e-04
Epoch 92/500
34/34 [==============================] - 0s - loss: 3.2749e-04 - mean_squared_error: 3.2749e-04
Epoch 93/500
34/34 [==============================] - 0s - loss: 3.1385e-04 - mean_squared_error: 3.1385e-04
Epoch 94/500
34/34 [==============================] - 0s - loss: 3.0065e-04 - mean_squared_error: 3.0065e-04
Epoch 95/500
34/34 [==============================] - 0s - loss: 2.8760e-04 - mean_squared_error: 2.8760e-04
Epoch 96/500
34/34 [============

34/34 [==============================] - 0s - loss: 1.9858e-05 - mean_squared_error: 1.9858e-05
Epoch 162/500
34/34 [==============================] - 0s - loss: 1.8987e-05 - mean_squared_error: 1.8987e-05
Epoch 163/500
34/34 [==============================] - 0s - loss: 1.8149e-05 - mean_squared_error: 1.8149e-05
Epoch 164/500
34/34 [==============================] - 0s - loss: 1.7356e-05 - mean_squared_error: 1.7356e-05
Epoch 165/500
34/34 [==============================] - 0s - loss: 1.6608e-05 - mean_squared_error: 1.6608e-05
Epoch 166/500
34/34 [==============================] - 0s - loss: 1.5898e-05 - mean_squared_error: 1.5898e-05
Epoch 167/500
34/34 [==============================] - 0s - loss: 1.5206e-05 - mean_squared_error: 1.5206e-05
Epoch 168/500
34/34 [==============================] - 0s - loss: 1.4555e-05 - mean_squared_error: 1.4555e-05
Epoch 169/500
34/34 [==============================] - 0s - loss: 1.3923e-05 - mean_squared_error: 1.3923e-05
Epoch 170/500
34/34 [===

34/34 [==============================] - 0s - loss: 4.6108e-07 - mean_squared_error: 4.6108e-07
Epoch 236/500
34/34 [==============================] - 0s - loss: 4.3512e-07 - mean_squared_error: 4.3512e-07
Epoch 237/500
34/34 [==============================] - 0s - loss: 4.1050e-07 - mean_squared_error: 4.1050e-07
Epoch 238/500
34/34 [==============================] - 0s - loss: 3.8721e-07 - mean_squared_error: 3.8721e-07
Epoch 239/500
34/34 [==============================] - 0s - loss: 3.6502e-07 - mean_squared_error: 3.6502e-07
Epoch 240/500
34/34 [==============================] - 0s - loss: 3.4367e-07 - mean_squared_error: 3.4367e-07
Epoch 241/500
34/34 [==============================] - 0s - loss: 3.2357e-07 - mean_squared_error: 3.2357e-07
Epoch 242/500
34/34 [==============================] - 0s - loss: 3.0510e-07 - mean_squared_error: 3.0510e-07
Epoch 243/500
34/34 [==============================] - 0s - loss: 2.8779e-07 - mean_squared_error: 2.8779e-07
Epoch 244/500
34/34 [===

...

34/34 [==============================] - 0s - loss: 9.7880e-07 - mean_squared_error: 9.7880e-07
Epoch 160/500
34/34 [==============================] - 0s - loss: 8.7146e-07 - mean_squared_error: 8.7146e-07
Epoch 161/500
34/34 [==============================] - 0s - loss: 7.7183e-07 - mean_squared_error: 7.7183e-07
Epoch 162/500
34/34 [==============================] - 0s - loss: 6.8260e-07 - mean_squared_error: 6.8260e-07
Epoch 163/500
34/34 [==============================] - 0s - loss: 6.0620e-07 - mean_squared_error: 6.0620e-07
Epoch 164/500
34/34 [==============================] - 0s - loss: 5.3889e-07 - mean_squared_error: 5.3889e-07
Epoch 165/500
34/34 [==============================] - 0s - loss: 4.8041e-07 - mean_squared_error: 4.8041e-07
Epoch 166/500
34/34 [==============================] - 0s - loss: 4.2736e-07 - mean_squared_error: 4.2736e-07
Epoch 167/500
34/34 [==============================] - 0s - loss: 3.7831e-07 - mean_squared_error: 3.7831e-07
Epoch 168/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.7674e-10 - mean_squared_error: 1.7674e-10
Epoch 234/500
34/34 [==============================] - 0s - loss: 1.6649e-10 - mean_squared_error: 1.6649e-10
Epoch 235/500
34/34 [==============================] - 0s - loss: 1.5796e-10 - mean_squared_error: 1.5796e-10
Epoch 236/500
34/34 [==============================] - 0s - loss: 1.5128e-10 - mean_squared_error: 1.5128e-10
Epoch 237/500
34/34 [==============================] - 0s - loss: 1.4076e-10 - mean_squared_error: 1.4076e-10
Epoch 238/500
34/34 [==============================] - 0s - loss: 1.3100e-10 - mean_squared_error: 1.3100e-10
Epoch 239/500
34/34 [==============================] - 0s - loss: 1.2040e-10 - mean_squared_error: 1.2040e-10
Epoch 240/500
34/34 [==============================] - 0s - loss: 1.1032e-10 - mean_squared_error: 1.1032e-10
Epoch 241/500
34/34 [==============================] - 0s - loss: 1.0066e-10 - mean_squared_error: 1.0066e-10
Epoch 242/500
34/34 [===

34/34 [==============================] - 0s - loss: 5.8213e-14 - mean_squared_error: 5.8213e-14
Epoch 308/500
34/34 [==============================] - 0s - loss: 7.4527e-14 - mean_squared_error: 7.4527e-14
Epoch 309/500
34/34 [==============================] - 0s - loss: 6.8001e-14 - mean_squared_error: 6.8001e-14
Epoch 310/500
34/34 [==============================] - 0s - loss: 6.2527e-14 - mean_squared_error: 6.2527e-14
Epoch 311/500
34/34 [==============================] - 0s - loss: 5.4855e-14 - mean_squared_error: 5.4855e-14
Epoch 312/500
34/34 [==============================] - 0s - loss: 5.3555e-14 - mean_squared_error: 5.3555e-14
Epoch 313/500
34/34 [==============================] - 0s - loss: 4.9944e-14 - mean_squared_error: 4.9944e-14
Epoch 314/500
34/34 [==============================] - 0s - loss: 5.0197e-14 - mean_squared_error: 5.0197e-14
Epoch 315/500
34/34 [==============================] - 0s - loss: 4.2734e-14 - mean_squared_error: 4.2734e-14
Epoch 316/500
34/34 [===

34/34 [==============================] - 0s - loss: 2.1802e-14 - mean_squared_error: 2.1802e-14
Epoch 382/500
34/34 [==============================] - 0s - loss: 1.1890e-14 - mean_squared_error: 1.1890e-14
Epoch 383/500
34/34 [==============================] - 0s - loss: 1.0623e-14 - mean_squared_error: 1.0623e-14
Epoch 384/500
34/34 [==============================] - 0s - loss: 1.6154e-14 - mean_squared_error: 1.6154e-14
Epoch 385/500
34/34 [==============================] - 0s - loss: 1.3908e-14 - mean_squared_error: 1.3908e-14
Epoch 386/500
34/34 [==============================] - 0s - loss: 1.5527e-14 - mean_squared_error: 1.5527e-14
Epoch 387/500
34/34 [==============================] - 0s - loss: 1.5741e-14 - mean_squared_error: 1.5741e-14
Epoch 388/500
34/34 [==============================] - 0s - loss: 1.6708e-14 - mean_squared_error: 1.6708e-14
Epoch 389/500
34/34 [==============================] - 0s - loss: 1.3939e-14 - mean_squared_error: 1.3939e-14
Epoch 390/500
34/34 [===

34/34 [==============================] - 0s - loss: 5.2283e-15 - mean_squared_error: 5.2283e-15
Epoch 456/500
34/34 [==============================] - 0s - loss: 5.2723e-15 - mean_squared_error: 5.2723e-15
Epoch 457/500
34/34 [==============================] - 0s - loss: 4.6715e-15 - mean_squared_error: 4.6715e-15
Epoch 458/500
34/34 [==============================] - 0s - loss: 1.1052e-14 - mean_squared_error: 1.1052e-14
Epoch 459/500
34/34 [==============================] - 0s - loss: 1.1026e-14 - mean_squared_error: 1.1026e-14
Epoch 460/500
34/34 [==============================] - 0s - loss: 1.1759e-14 - mean_squared_error: 1.1759e-14
Epoch 461/500
34/34 [==============================] - 0s - loss: 1.2001e-14 - mean_squared_error: 1.2001e-14
Epoch 462/500
34/34 [==============================] - 0s - loss: 1.1955e-14 - mean_squared_error: 1.1955e-14
Epoch 463/500
34/34 [==============================] - 0s - loss: 5.3523e-15 - mean_squared_error: 5.3523e-15
Epoch 464/500
34/34 [===

### Mean Aggregation

In [11]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
            



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (15, 30)
Getting op from chain node  1
Shapes:
 X = (15, 31)
Getting op from chain node  2
Shapes:
 X = (15, 32)
Getting op from chain node  3
Shapes:
 X = (15, 33)
Getting op from chain node  4
Shapes:
 X = (15, 34)
Getting op from chain node  5
Shapes:
 X = (15, 35)


ENSEMBLE 2:


Getting op from chain node  0
Shapes:
 X = (15, 30)
Getting op from chain node  1
Shapes:
 X = (15, 31)
Getting op from chain node  2
Shapes:
 X = (15, 32)
Getting op from chain node  3
Shapes:
 X = (15, 33)
Getting op from chain node  4
Shapes:
 X = (15, 34)
Getting op from chain node  5
Shapes:
 X = (15, 35)


ENSEMBLE 3:


Getting op from chain node  0
Shapes:
 X = (15, 30)
Getting op from chain node  1
Shapes:
 X = (15, 31)
Getting op from chain node  2
Shapes:
 X = (15, 32)
Getting op from chain node  3
Shapes:
 X = (15, 33)
Getting op from chain node  4
Shapes:
 X = (15, 34)
Getting op from chain node  5
Shapes:
 X = (15, 35)


ENSEMBLE 4:




In [12]:
##Calculating the mean
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [13]:
#Adding all results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2))

### Evaluating Mean Aggregation

In [14]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))

RMSE:  2.7544480471039736
MAE:  1.3966327896118165
R_Squared:  0.8067305843390556
Adjusted R-squared -  0.48461489157081505
